In [1]:
import importlib.util
def module_from_file(module_name, file_path): #The code that imports the file which originated the training with all the instructions
            spec = importlib.util.spec_from_file_location(module_name, file_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)
            return module

from importlib import import_module
#foo = import_module(fold_folder+'/Funs.py', package=None)


folder = './xforanalogs/NA24by48/Z8/yrs500/interT15fw20.1.20lrs4'
foo = module_from_file("foo", f'{folder}/Funs.py')

import random as rd  
from scipy.stats import norm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.metrics import log_loss

tff = foo.tff # tensorflow routines 
ut = foo.ut # utilities
ln = foo.ln #Learn2_new.py
print("==Checking GPU==")
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

print("==Checking CUDA==")
tf.test.is_built_with_cuda()

#from tensorflow.keras.preprocessing.image import ImageDataGenerator
sys.path.insert(1, '../ERA')

print("==Reading data==")

year_permutation = np.load(f'{folder}/year_permutation.npy')

2022-07-05 15:43:19.370375: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
In this environment you cannot import Basemap
In this environment you cannot import Basemap


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-07-05 15:43:22.612132: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-05 15:43:22.617793: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-05 15:43:22.829051: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-05 15:43:22.829107: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (apollo2048g): /proc/driver/nvidia/version does not exist
2022-07-05 15:43:22.829131: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices

==Checking GPU==
==Checking CUDA==
==Reading data==


In [2]:
run_vae_kwargs = ut.json2dict(f"{folder}/config.json")
T = ut.extract_nested(run_vae_kwargs, 'T')
if (ut.keys_exists(run_vae_kwargs, 'label_period_start') and ut.keys_exists(run_vae_kwargs, 'label_period_end')):
    label_period_start = ut.extract_nested(run_vae_kwargs, 'label_period_start')
    label_period_end = ut.extract_nested(run_vae_kwargs, 'label_period_end')
    time_start = ut.extract_nested(run_vae_kwargs, 'time_start')
    time_end = ut.extract_nested(run_vae_kwargs, 'time_end')

In [3]:
n_days = time_end-time_start-T+1
n_days

105

In [4]:
import pickle
fold = 0
open_file = open(f'{folder}/fold_{fold}/analogues.pkl', "rb")
analogues = pickle.load(open_file)
open_file.close()

checkpoint = 10

analogues['ind_new_tr'][checkpoint]//n_days

array([[  0, 241,   0, ..., 351,   3,  52],
       [  0,   0,  54, ..., 102, 145, 176],
       [  0,   0, 405, ..., 390, 251, 378],
       ...,
       [449,  99,  99, ...,  74, 255, 123],
       [281, 177, 241, ..., 208, 313, 198],
       [208, 241, 211, ..., 376, 209, 339]])

In [5]:
analogues['ind_new_tr'][checkpoint]%n_days

array([[  0,  12,   1, ...,  11,  48,  38],
       [  1,   0,  29, ..., 100,  31,  87],
       [  2,   3,  20, ...,  87,  93,  24],
       ...,
       [101,  44,  45, ...,  66,  59,  95],
       [ 70,  46,  21, ...,  75,  24,  30],
       [ 81,  21,  88, ...,  73,  25,  33]])

In [6]:
analogues['ind_new_tr'][checkpoint][:9,:9]

array([[    0, 25317,     1, 25055, 10706, 32025, 32026, 32027,  5699],
       [    1,     0,  5699, 42841,     2,  5701, 42840, 27301,  5700],
       [    2,     3, 42545,  5363,  5365,  5699, 42841, 42544,     1],
       [    3,     2,     4, 42545,  5367,  5366,  5365,  5368, 42546],
       [    4,     5,     3,  5368,  5367, 12291,  5361,  5366, 32557],
       [    5,     4,     6,  5368, 47154,  5361,  5367, 32557,     3],
       [    6, 47154,     5,     7, 26782, 23223, 32562, 32561, 47155],
       [    7,     8,  4332, 26782, 32562, 42319,  4331,     6, 47155],
       [    8, 42320, 34685, 27246, 42014, 27245, 47156, 21432, 34236]])

In [7]:
import numpy as np
#selfanalogues = (analogues['ind_new_tr'][checkpoint]//n_days)[:,0][np.newaxis].T
selfanalogues = ((np.arange(analogues['ind_new_tr'][checkpoint].shape[0]//n_days)[np.newaxis].T*np.ones((analogues['ind_new_tr'][checkpoint].shape[0]//n_days,n_days))).astype(int)).flatten()[np.newaxis].T # This is a matrix of years related to the raw index
#selfanalogues*np.ones((selfanalogues.shape[0],analogues['ind_new_tr'][checkpoint].shape[1]))
sameyear = selfanalogues*np.ones((selfanalogues.shape[0],analogues['ind_new_tr'][checkpoint].shape[1]))==analogues['ind_new_tr'][checkpoint]//n_days # A conditional matrix showing if the entry belongs to the same year 
#sameyear
noselfanalogs = (np.where(sameyear,-1,analogues['ind_new_tr'][checkpoint])) # We set to -1 all the entries that are analogs of the same year
print(np.mean(noselfanalogs == -1))
noselfanalogs

0.011160084656084656


array([[   -1, 25317,    -1, ..., 36866,   363,  5498],
       [   -1,    -1,  5699, ..., 10810, 15256, 18567],
       [   -1,    -1, 42545, ..., 41037, 26448, 39714],
       ...,
       [   -1, 10439, 10440, ...,  7836, 26834, 13010],
       [29575, 18631, 25326, ..., 21915, 32889, 20820],
       [21921, 25326, 22243, ..., 39553, 21970, 35628]])

In [8]:
noselfanalogsmoved = []
for noselfanalogs_row in noselfanalogs: # loop over samples
    temp = np.delete(noselfanalogs_row,noselfanalogs_row==-1) # remove the values equal to -1 
    noselfanalogsmoved.append(np.pad(temp, (0,noselfanalogs_row.shape[0] - temp.shape[0]), constant_values=(noselfanalogs.shape[0],noselfanalogs.shape[0])))  # pad with the length of the time series so that if we accidently get such analog the error will be returned
noselfanalogsmoved = np.array(noselfanalogsmoved)
noselfanalogsmoved

array([[25317, 25055, 10706, ..., 47250, 47250, 47250],
       [ 5699, 42841,  5701, ..., 47250, 47250, 47250],
       [42545,  5363,  5365, ..., 47250, 47250, 47250],
       ...,
       [10439, 10440, 26960, ..., 47250, 47250, 47250],
       [29575, 18631, 25326, ..., 20820, 47250, 47250],
       [21921, 25326, 22243, ..., 39553, 21970, 35628]])

In [9]:
analogues['ind_new_va'][checkpoint].shape

(5250, 1000)

In [10]:
for j in range(5+np.max([0,3])//3):
    print(j)

0
1
2
3
4
5
